# Intro

This notebook describes __compiler-only__ features that help to enforce types in sophisticated ways but don't actually affect the output.

A lot of it is built on syntax that is based on syntax from real code but means something different when used just on types.

What it usually comes down to is __defining new types__ based on other types and literal values.

# Type vs. Interface

`type` and `interface` are similar concepts in that they are both __compiler-only__ features, whereas classes actually generate a constructor function just by being defined.

Here are some key differences:
- `interface` is for class-like declarations while `type` is used more like a variable
  - this difference is the reason for a lot of the next few differences
- `interface` uses `extends` while `type` uses _unions_
- __declaration merging__ is only for `interface`
- `type` can also hold __literal values__ (while `interface` cannot)

`type` is the more useful construct for __metaprogramming__ in particular.

In [5]:
(() => {
    interface MyType {
        a: number;
    }
    
    type MyType2 = {  // note the = here
        a: number;
    }
    
    type MyType3 = string;
    type MyType4 = string | number;
})();

undefined

# Literal Values

The compiler can take literal values as if they were types, and enforce them at the compiler level.

Note that this __does not do runtime checking__ on any values.

In [14]:
(() => {
    type T1 = string;
    type T2 = 'hi';
    type T3 = 0 | 'yo';
    
    // const x: 0 | 'yo' = 1; // ILLEGAL
    const x: 0 | 'yo' = 'yo';
    const y: T3 = 0;
    
    type Config = {
        mode: 'light'|'dark';
        version: 0|1;
    }
    // const c: Config = {mode: 'grey', version: 2}; // ILLEGAL
    
    interface MyInterface {
        mode: 'light'|'dark'; // allowed because it's the field that's being typed
    }
})();

undefined

# Index Signatures

You can have whatever and however many fields you want in the literal object, as long as the types match the signature.

This is covered in another notebook in more detail.

In [25]:
(() => {
    type Horse = {horseName: string};
    
    type OnlyBoolsAndHorses = {
        [key: string]: boolean | Horse;
    };

    const x: OnlyBoolsAndHorses = {
        a: true,
        b: {horseName: 'bla'},
        //c: {}, // ILLEGAL
        //d: 'hi', // ILLEGAL
        arbitraryName: false,
    };
})();

undefined

# Call Signatures

Covered elsewhere, but as a reminder, these let you specify an interface for something that's callable like a function. JavaScript doesn't support defining classes that can do that, but you can use it for things like function variables that behave like that intrinsically.

# Construct Signatures

Covered elsewhere - listing here for completeness.

# typeof

This is covered elsewhere, but as a reminder, this gets you the compile-time type of a variable.

# keyof

Gives you the __union of literal key names__ for a type.

Note that if you have an __index signature__, the compiler is smart enough to use the general type union instead of literals. For string keys, the type will be `number|string` due to JS accepting number indices for string keys.

`keyof` only considers __public members__.  It __includes methods__, so be careful.

In [30]:
(() => {
    interface Point {
        x: number;
        y: number;
        name: string;
    }
    type K = keyof Point;
    
    const x: K = 'x';
    const y: K = 'name';
    // const z: K = 'bla'; // not one of the names of the keys of Point
    // const z2: K = 0; // not the type of a key of Point
})();

undefined

In [36]:
(() => {
    class MyClass {
        x: number;
        private y: number;
        protected z: number;
    }
    
    type K = keyof MyClass;
    const x: K = 'x';
    // const y: K = 'y'; // not a public key
    // const z: K = 'z'; // not a public key
})();

undefined

In [41]:
(() => {
    class Point {
        x: number;
        y: number;
        z: number;
    }
    
    function set(point: Point, field: keyof Point, value: number) {
        point[field] = value;
    }
    
    const p = {x: 100, y: 100, z: 100};
    set(p, 'y', 200);
    console.log(p);
})();

{ x: 100, y: 200, z: 100 }


undefined

# Indexed Access Types

For getting the __type of a field__ of an object.

The syntax is based on indexing objects, but with some enhancements to support __unions__.

In general, you can use `[]` with a __keyname or a union of keynames__ (including as retrieved by `keyof`) to get a value type (or union thereof).  You can also __use a type index__ to get all the value types that would retrieve (as in the array example).

In [46]:
(() => {
    type Person = {
        age: number,
        name: string,
        alive: boolean,
    };
    
    type TNumber = Person['age'];
    type TNumberOrString = Person['age' | 'name'];  // union of types of these keys
    type TNumberOrStringOrBoolean = Person[keyof Person]; // union of types of all public fields
    
    const arr = [1, 2, 3, 'hi'];
    type TArr = typeof arr[number]; // type of anything that would be indexed by number (string | number)
})();

undefined

# extends

Although it uses the same keyword as class/interface inheritence, `extends` in the context of __type constraints__ just means it has the members of that class.  Think of it more like __structural compatibility__.

It works for __primitive types__ too (and is the way to specify you need a specific primitive type).

You can use it with __unions__ to support multiple input types in the constraint.  Just keep in mind the type can __only be one thing__ in a given call!

In [59]:
(() => {
    type Point = {x: number, y: number};
    type Point3D = {x: number, y: number, z: number};
    
    const p: Point3D = {x: 100, y: 200, z: 300};
    let q: Point = p;
    
    function getX<T extends Point>(p: T) {
        return p.x;
    }
    
    console.log(getX(p));
    console.log(getX(q));
    
    function getSquare<T extends number>(n: T) {
        return n * n;
    }
    console.log(getSquare(100));
    
    function doStuff< T extends number|string>(n: T, o: T) {
        console.log(n, o);
    }
    // doStuff('yo', 3); // mismatched types
    doStuff('hi', 'there');
    
})();

100
100
10000
hi there


undefined

# Generic Types

To directly specify the type of a generic function, you hadd `<>` with the type arg(s) and constraints __to the left__ of the args list, and then you can use the type arg(s) in the rest of the specification.

Just like with function params, the structure is checked, not the specific names.  So the type args and the param names can mismatch.

You can use the generic type as a __call signature__ (which effectively means the same thing) by enclosing with `{}` and using `:` as shown below.  By making the `{}` an `interface`, you are defining a __generic interface__, which is not the same thing as an interface that takes a generic type arg (which is also allowed).

In [76]:
(() => {
    // generic function
    function f<T extends number|string>(x: T, y: T) {
        console.log(x, y);
    }
    f(100, 200);
    
    // we know we can do this
    type F = typeof f;
    
    // but what if we wanted to specify what F looks like directly?
    type G = <TT extends number|string>(xx: TT, y: TT) => void; // note the name mismatches are ok
    const g: G = f; // compatible!
    
    // enclose in {} and use : instead of =>
    type H = {<TT extends number|string>(xx: TT, y: TT): void};
    
    // generic interface
    interface GI {
        <TT extends number|string>(xx: TT, y: TT): void;
    }
    const gi: GI = f;
    
    // interface that is a generic (different)
    interface IG<T> {
        x: T;
    }
})();

100 200


undefined

# Conditional Types

In the simple case, this leverages __type constraint__ syntax in combination with __ternary operators__.

You can also use `<>` on the __leftside__ of `=` to effectively make a function that operates on types at compile-time. Two examples are given below, one using `extends` and the other not using it (to show the generality of the concept). This can be used to __eliminate overloads__. You can constraint the type parameters passed in to the metafunction by putting another `extends` on the leftside.

The `infer` keyword can be used to pull a type argument out of a type passed in, as shown in 3rd cell below.

These kinds of operations are __distributive__ across __unions__, so the result is like applying the "function" to each item in the union and then taking a union of the result.  There is a special syntax involving `[]` to disable that if needed for some reason.  Remember that `keyof` returns a union!

TypeScript has a lot of built-in __utility types__ that make use of these capabilities (eg. `ReturnType<>`).

In [77]:
(() => {
    type MyBaseClass = {};
    type MyOtherClass = {x: number}; // this does extend MyBaseClass by type constraint rules
    
    type T = MyOtherClass extends MyBaseClass ? string : number; // will be string
    const t: T = 'hi';
})();

undefined

In [86]:
(() => {
    // StringOrInt is a function that conditionally branches its return type here
    type StringOrInt<T> = T extends string ? string : number;
    const x: StringOrInt<number> = 50;
    const y: StringOrInt<string> = 's';
    // const z: StringOrInt = 100; // ILLEGAL (won't infer from this)
    
    // MyType is a function that gets a union of its arguments keynames here
    type MyType<T> = keyof T;
    let m: MyType<{x: number}> = 'x';
    
    // MyType2 is a function with the allowed passed-in type constrained
    type MyType2<T extends number> = T extends number ? number : string;
})();

undefined

In [88]:
(() => {
    type Flatten<T> = T extends Array<infer Item> ? Item : T;
    
    type U = Flatten<Array<number>>; // item type pulled out
    type V = Flatten<number>; // not array, so just uses the type passed in
    
    const u: U = 50;
    const v: V = 50;
})();

undefined

In [91]:
(() => {
    type Flatten<T> = T extends Array<infer Item> ? Item : T;
    type U = Flatten<Array<number>|Array<string>|boolean>; // number|boolean|string
    
    const u: U = 10;
    const v: U = 'hi';
    const w: U = true;
})();

undefined

# Mapped Types

These allow you to __transform the types of properties__ of one type to make a new type.

It uses the same concept of `<>` on the left to be like a meta-function as conditional types above.

`readonly` and `?` modifiers in the original type are directly copied to the new mapped type, as shown in the 2nd example.  There is special syntax with `+` and `-` to add or remove one or both of those for all members to make them uniform if needed.

There are __many more features__ of mapped types than are listed here - see docs if need more stuff.

In [95]:
(() => {
    type AllAsBool<Type> = {
        [Property in keyof Type]: boolean; // all properties of the input type are mapped to booleans
    };
    
    class OriginalType {
        x: number;
        y: number;
    }
    
    type NewType<T> = {
        [Whatever in keyof T]: boolean;    // like a 'for' loop
    }
    
    const n: NewType<OriginalType> = {x: true, y: false}; // same keys, but boolean values
})();

undefined

In [101]:
(() => {
    type AllAsBool<Type> = {
        [Property in keyof Type]: boolean; // all properties of the input type are mapped to booleans
    };
    
    class OriginalType {
        x?: number;
        readonly y: number;
    }
    
    type NewType<T> = {
        [Whatever in keyof T]: boolean;
    }
    
    const n: NewType<OriginalType> = {y: true};
    // n.y = false;  // ILLEGAL because readonly
})();

undefined

# Interpolated Literal Types

__String interpolation syntax__ can be used on types that contain literals to generate new literal types.

__Unions are cross-multiplied__ to produce all the combinations.

Built-in __utility types__ like `Lowercase<>` can transform literal types at compile-time.

There are some complex/sophisticated examples in the docs. Remember that `keyof` makes a union.

In [103]:
(() => {
    type S = 'hi';
    type T = `${S}, Bob!`
    
    const t: T = 'hi, Bob!';
})();

undefined

In [104]:
(() => {
    type S = 'hi'|'bye';
    type T = `${S}, bob!` | `${S}, alice!`;
    
    const t1: T = 'hi, bob!';
    const t2: T = 'hi, alice!';
    const t3: T = 'bye, bob!';
    const t4: T = 'bye, alice!';
})();

undefined

In [107]:
(() => {
    type T = Lowercase<'aBc'>;
    const t: T = 'abc';
})();

undefined

In [113]:
(() => {
    class OriginalType {
        x: number;
        y: number;
        z: number;
    }
    
    type K = keyof OriginalType;
    type NewType = `Hello, ${K}`;
    type N = NewType;
    
    const n: N = 'Hello, x';
    const o: N = 'Hello, y';
    const p: N = 'Hello, z';
})();

undefined

# Discriminated Unions

This fancy sounding name just means __type narrowing__ can be done by the compiler based on __literal values__ within an interface.

In [123]:
(() => {
    interface Interface1 {
        x: number;
        tag: 'abc';  // the overlapping (differing) literal (that you can discriminate with)
    }
    interface Interface2 {
        s: string;
        tag: 'def';
    }
    type T = Interface1 | Interface2; // the "union" part
    
    const o: T = { // o is of the type of the union, but it has a specific tag
        x: 100,
        tag: 'abc',
    }
    
    function f(t: T) {
        if (o.tag === 'abc') {
            console.log(o.x); // even though you used the union, you can access non-overlapping members
        }
        if (o.tag === 'def') {
            console.log(o.s); // ditto
        }
    }
    
    f(o);
})();

100


undefined

# Type Intersections

The `&` operator on types creates a new type that __must have all members from both__ types.

In a sense, it gets the __union of properties__, whereas type unions get the __intersection of properties__.

In [125]:
(() => {
    class MyClass {
        x: number;
        s: string;
    }
    class OtherClass {
        x: number;
        u: string;
    }
    
    type T = MyClass & OtherClass;
    
    const t: T = {x: 100, s: 'hi', u: 'bye'};
})();

undefined

# Utility Types

https://www.typescriptlang.org/docs/handbook/utility-types.html